In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


sigmoid activation function and its derivative 
soft max activation function for multi class output

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))


def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)



initialising weights and biases

In [ ]:
def initialize_parameters(input_size, hidden_size, binary_output_size, class_output_size):
    np.random.seed(42)
    W1_binary = np.random.randn(input_size, hidden_size) * 0.01 
    b1_binary = np.zeros((1, hidden_size))         
    W1_class = np.random.randn(input_size, hidden_size) * 0.01 
    b1_class = np.zeros((1, hidden_size))                            
    W2_binary = np.random.randn(hidden_size, binary_output_size) * 0.01
    b2_binary = np.zeros((1, binary_output_size))
    W2_class = np.random.randn(hidden_size, class_output_size) * 0.01
    b2_class = np.zeros((1, class_output_size))
    return W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class 




defining forward and backward propagation and computing the losses

In [ ]:
def forward_propagation(X, W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class):
    Z1_binary = np.dot(X, W1_binary) + b1_binary
    A1_binary = sigmoid(Z1_binary)

    Z1_class = np.dot(X,W1_class)+b1_class
    A1_class = sigmoid(Z1_class)


    Z2_binary = np.dot(A1_binary, W2_binary) + b2_binary
    A2_binary = sigmoid(Z2_binary)

    Z2_class = np.dot(A1_class, W2_class) + b2_class
    A2_class = softmax(Z2_class)

    cache = {"Z1_binary": Z1_binary, "A1_binary":  A1_binary,"Z1_class": Z1_class,"A1_class":A1_class, "Z2_binary": Z2_binary, "A2_binary": A2_binary, "Z2_class": Z2_class, "A2_class": A2_class}
    return A2_binary, A2_class, cache


def compute_binary_loss(y_binary, A2_binary):
    m = y_binary.shape[0]
    loss = -np.mean(y_binary * np.log(A2_binary) + (1 - y_binary) * np.log(1 - A2_binary))
    return loss

def compute_class_loss(y_class, A2_class):
    m = y_class.shape[0]
    loss = -np.mean(np.sum(y_class * np.log(A2_class), axis=1))
    return loss

def backward_propagation(X, y_binary, y_class, cache, W2_binary, W2_class):
    m = X.shape[0]
    A1_binary = cache["A1_binary"]
    A1_class = cache["A1_class"]
    A2_binary = cache["A2_binary"]
    A2_class = cache["A2_class"]

    dZ2_binary = A2_binary - y_binary
    dW2_binary = np.dot(A1_binary.T, dZ2_binary) / m
    db2_binary = np.sum(dZ2_binary, axis=0, keepdims=True) / m

    dZ2_class = A2_class - y_class
    dW2_class = np.dot(A1_class.T, dZ2_class) / m
    db2_class = np.sum(dZ2_class, axis=0, keepdims=True) / m

    dZ1_binary = np.dot(dZ2_binary, W2_binary.T)
    dZ1_class = np.dot(dZ2_class, W2_class.T)
    
    dZ1_binary *= sigmoid_derivative(cache["Z1_binary"])
    dZ1_class *= sigmoid_derivative(cache["Z1_class"])
    
    dW1_binary = np.dot(X.T, dZ1_binary) / m
    db1_binary = np.sum(dZ1_binary, axis=0, keepdims=True) / m
    
    dW1_class = np.dot(X.T, dZ1_class) / m
    db1_class = np.sum(dZ1_class, axis=0, keepdims=True) / m

    gradients = {
        "dW1_binary": dW1_binary,
        "db1_binary": db1_binary,
        "dW1_class": dW1_class,
        "db1_class": db1_class,
        "dW2_binary": dW2_binary,
        "db2_binary": db2_binary,
        "dW2_class": dW2_class,
        "db2_class": db2_class
    }
    
    return gradients

updating the parameters

In [ ]:
def update_parameters(W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class, gradients, learning_rate):

    W1_binary -= learning_rate * gradients["dW1_binary"]
    b1_binary -= learning_rate * gradients["db1_binary"]
    W1_class -= learning_rate * gradients["dW1_class"]
    b1_class -= learning_rate * gradients["db1_class"]
    W2_binary -= learning_rate * gradients["dW2_binary"]
    b2_binary -= learning_rate * gradients["db2_binary"]
    W2_class -= learning_rate * gradients["dW2_class"]
    b2_class -= learning_rate * gradients["db2_class"]
    return W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class



defining to train the dats set 

In [ ]:
def train_neural_network(X, y_binary, y_class, hidden_size=64, learning_rate=0.01, epochs=100):
    input_size = X.shape[1]
    binary_output_size = 1
    class_output_size = y_class.shape[1]

    W1_binary, b1_binary, W1_class,b1_class,W2_binary, b2_binary, W2_class, b2_class = initialize_parameters(input_size, hidden_size, binary_output_size, class_output_size)
    losses = []

    for _ in range(epochs):

        A2_binary, A2_class, cache = forward_propagation(X, W1_binary, b1_binary, W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class)

        binary_loss = compute_binary_loss(y_binary, A2_binary)
        class_loss = compute_class_loss(y_class, A2_class)
        total_loss = binary_loss + class_loss
        losses.append(total_loss)

        gradients = backward_propagation(X, y_binary, y_class, cache, W2_binary, W2_class)

        W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class = update_parameters(W1_binary, b1_binary,W1_class , b1_class, W2_binary, b2_binary, W2_class, b2_class, gradients, learning_rate)

                
    
    plt.plot(losses)
    plt.xlabel('Epochs')
    plt.ylabel('Total Loss')
    plt.title('Cost Convergence')
    plt.show()

    

    return W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class


predicting the testing data set 

In [ ]:
def predict(X, W1_binary, b1_binary, W1_class,b1_class,W2_binary, b2_binary, W2_class, b2_class):
    A2_binary, A2_class, _ = forward_propagation(X, W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class)
    binary_predictions = (A2_binary > 0.5).astype(int)
    class_predictions = np.argmax(A2_class, axis=1)
    return binary_predictions, class_predictions

metrics

In [ ]:
def f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    return 2 * (precision * recall) / (precision + recall + 1e-8)

training the data set

In [ ]:
file_path =  r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\nn_train.csv"  
data = pd.read_csv(file_path).drop(columns=['ID'])

X = data.iloc[:, :-2].values 
y_binary = data.iloc[:, -2].values.reshape(-1, 1)  
y_class = pd.get_dummies(data.iloc[:, -1]).values  

    
np.random.seed(42)
indices = np.random.permutation(len(X))
train_size = int(0.8 * len(X))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

X_train, X_test = X[train_indices], X[test_indices]

X_train_mean=X_train.mean()
X_train_std =X_train.std()                       #normalising the data       
X_train=(X_train-X_train_mean)/X_train_std
X_test = (X_test-X_train_mean)/X_train_std

y_binary_train, y_binary_test = y_binary[train_indices], y_binary[test_indices]
y_class_train, y_class_test = y_class[train_indices], y_class[test_indices]

W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class = train_neural_network(X_train, y_binary_train, y_class_train, hidden_size=128, learning_rate=0.8, epochs=7)

binary_predictions, class_predictions = predict(X_test, W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class)

binary_accuracy = np.mean(binary_predictions == y_binary_test)
class_predictions_one_hot = np.zeros_like(y_class_test)
class_predictions_one_hot[np.arange(len(class_predictions)), class_predictions] = 1
class_accuracy = np.mean(np.all(class_predictions_one_hot == y_class_test, axis=1))

binary_f1 = f1_score(y_binary_test.flatten(), binary_predictions.flatten())

print(f"Binary Accuracy: {binary_accuracy:.4f}")
print(f"Binary F1 Score: {binary_f1:.4f}")
print(f"Class Accuracy: {class_accuracy:.4f}")      

testing the data set

In [ ]:
test_data = pd.read_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\nn_test.csv")

X_new = test_data.drop(columns=["ID"]).values

X_new = (X_new - X_train_mean)/X_train_std 

binary_new_pred, class_new_pred = predict(X_new, W1_binary, b1_binary,W1_class,b1_class, W2_binary, b2_binary, W2_class, b2_class)


test_data['binary_Predicted'] = binary_new_pred 
test_data['class_predicted'] =  class_new_pred


test_data.to_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\nn_test_predicted.csv", index=False)

print("Predictions saved in the original test data file.")
